### Agent #1:
##### Goal:
 a time series fetcher agent
##### Output: 

Normalized df





In [ ]:
# Required installs

# !pip install yfinance

In [1]:
#Imports
import pandas as pd
from __future__ import annotations
import yfinance as yf


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
def fetch_prices_yf (symbol: str, period: str ="1y", interval: str = "1d") -> pd.DataFrame:
    """
    Fetch OHLCV ('Open', 'High', 'Low', 'Close', 'Volume') time series from Yahoo Finance and return a normalized time series DataFrame.

        symbol (str): Stock ticker symbol. (TSLA for Tesla, AAPL for Apple, etc.)
        period (str): Data period to download ("1y" for 1 year)
        interval (str): Data interval (e.g., "1m" for one month and "1h" for 1 hour).

    """
    if not symbol or not isinstance(symbol,str):
        raise ValueError("Invalid symbol. Please provide a valid, non-empty stock ticker symbol as a string.")
    
    raw=yf.download(symbol, period=period, interval=interval , progress=False) # set progress to False to avoid printing download progress
    if raw is None or raw.empty:
        raise RuntimeError(f"Failed to fetch data for symbol: {symbol}")

    #normalize coloumns and index
    df = raw.reset_index()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]

    ## Ensure timestamp is standardized
    ts_col="Datetime" if "Datetime" in df.columns else "Date"
    df=df.rename(columns={ts_col:"timestamp"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    #Keep the relevant/expected columns only
    keep= ['time stamp','open','high', 'low', 'close', 'adj_close', 'volume']
    for k in keep:
        if k not in df.columns:
            df[k] =pd.NA
    df['symbol']=symbol.upper()

    #Sort + Drop Nulls
    df=df[keep+['symbol']].sort_values('timestamp').dropna(subset=['close']).reset_index(drop=True)



    return df